In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import rasterio as rio
from rasterio.windows import Window
import geopandas as gpd
import earthpy as et
import earthpy.spatial as es
import xarray as xr

In [6]:
from naip import *

In [7]:
filepath = '/home/jovyan/UrbanForest'
infile = os.path.join(filepath, 'ortho_1-1_hn_s_ca037_2016_1.tif')
outfile = os.path.join(filepath, 'ortho_1-1_hn_s_ca037_2016_1_ndvi.tif')

In [8]:
get_make_ndvi_args(infile, output='print')

RasterioIOError: TIFFReadDirectory:Failed to read directory at offset 370813140016